<a href="https://colab.research.google.com/github/AnikaZN/DS-Unit-2-Linear-Models/blob/master/Linear_Models_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [385]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv')
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,exclusive,terrace,loft,garden_patio,common_outdoor_space,wheelchair_access
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Train/Test Split

In [386]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)
df['created'].head()

0   2016-06-24 07:54:24
1   2016-06-12 12:19:27
2   2016-04-17 03:26:41
3   2016-04-18 02:22:02
4   2016-04-28 01:32:41
Name: created, dtype: datetime64[ns]

In [0]:
df['month'] = df['created'].dt.month

train = df[df['month'] < 6] #April and May
test = df[df['month'] == 6] #June

In [388]:
train.shape, test.shape

((31515, 35), (16785, 35))

In [389]:
train['price'].mean()

3432.7534190068222

# Baseline

In [0]:
from sklearn.metrics import mean_absolute_error as mae

In [391]:
y_test = test['price']
y_pred = [train['price'].mean()] * len(y_test)

print(len(y_test), len(y_pred))
print('Mean Absolute Error:', mae(y_test, y_pred))

16785 16785
Mean Absolute Error: 1052.6798017842948


# Features

In [392]:
df.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'exclusive', 'terrace', 'loft',
       'garden_patio', 'common_outdoor_space', 'wheelchair_access', 'month'],
      dtype='object')

In [0]:
from sklearn.linear_model import LinearRegression as linreg

In [394]:
model = linreg()
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
features = ['bedrooms',
           'bathrooms',
           'longitude', #approx 50 points
           'doorman', #approx 12 points
           'laundry_in_unit', #approx 4 points
           'interest_level', #approx 30 points
           'elevator'] #approx 3 points

target = 'price'

x_train = train[features]
y_train = train[target]

x_test = test[features]
y_test = test[target]

In [396]:
x_train.shape, y_train.shape

((31515, 7), (31515,))

# One-hot Encoding

In [0]:
#! pip install category_encoders

In [0]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)

x_train = encoder.fit_transform(x_train)
x_test = encoder.transform(x_test)

In [437]:
model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [438]:
model.coef_

array([ 552.52460057,  525.38591022, -359.88635605,  178.37638612,
        115.65748344, -111.84795398,  135.98218588,  -77.58008957,
         76.41203623])

In [439]:
model.intercept_

3432.7534190068222

# Scale

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [441]:
x_train

array([[-0.45807512, -0.42056347, -0.99016991, ..., -1.4773931 ,
        -0.55395212, -1.0537678 ],
       [-0.45807512, -0.42056347,  0.16078122, ...,  0.67686792,
        -0.55395212, -1.0537678 ],
       [ 2.28697859, -0.42056347,  0.7818223 , ...,  0.67686792,
        -0.55395212, -1.0537678 ],
       ...,
       [-0.45807512, -0.42056347, -0.59189356, ...,  0.67686792,
        -0.55395212,  0.94897567],
       [-1.37309302, -0.42056347, -1.27031344, ...,  0.67686792,
        -0.55395212,  0.94897567],
       [ 0.45694278, -0.42056347,  1.86526896, ...,  0.67686792,
        -0.55395212, -1.0537678 ]])

# MAE

In [442]:
y_pred = model.predict(x_test)
y_pred

array([3624.74088159, 3790.75899109, 2441.94926307, ..., 2598.84114865,
       3683.88376304, 3412.15024851])

In [443]:
mae(y_test, y_pred)

600.3931387569882